# Create synthetic data

Create synthetic data using:
* Headings from cleaned data extract from REDCap
* Dictionaries that were used to create the cleaned data extract and include definitions of what values are present
* Anticipated columns from council
* List of MSOAs in Northern Devon

## Set-up

### Packages, warnings and file paths

In [1]:
# Import required packages
from dataclasses import dataclass
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import random

In [2]:
# Ignore pandas warning as not relevant as not needing high performance
from warnings import simplefilter
simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [3]:
# Import functions defined elsewhere
from functions import calculate_scores

import sys
sys.path.append('../')
from utilities.response_labels import create_response_label_dict

In [4]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    data = '../data'
    headings = 'survey_data/headings.csv'
    msoa_shp = 'MSOA_2021_EW_BFC/MSOA_2021_EW_BFC_V6.shp'
    nd_msoa_shp = 'msoa_nd_2021/msoa_nd_2021.shp'
    synthetic_data = 'survey_data/synthetic_data_raw.csv'


paths = Paths()

### Define sample size

In [5]:
sample_size = 800
school_n = 7

## Create synthetic data

### Import headings

In [6]:
# Import headings from REDCap output to use to create synthetic data
headings = pd.read_csv(os.path.join(paths.data, paths.headings))

# Drop unneeded columns
headings = headings.drop(['Unnamed: 0', 'record_id', 'media_error'], axis=1).columns

# Add anticipated columns from council
headings = list(headings) + [
    'year_group', 'fsm', 'sen', 'ethnicity', 'english_additional', 'school']

# View head and tail
print(headings[:5])
print(headings[-5:])

['gender', 'transgender', 'sexual_orientation', 'neurodivergent', 'birth_parent1']
['fsm', 'sen', 'ethnicity', 'english_additional', 'school']


### Import dictionary of responses labels

This uses a function imported from utilities, as we use this dictionary in multiple places for the dashboard, so it makes more sense to define it in one place and pull from there.

In [7]:
labels = create_response_label_dict()
len(labels)

141

### Randomly sample values from dictionary to populate each column

In [8]:
# Identify measures that would be output by REDCap (drop those I calculated)
measures = [x for x in headings if ~np.logical_or(
    x.endswith('_score'), x.endswith('_lab'))]

# Initialise dataframe with row number matching sample size
data = pd.DataFrame(index=range(1, sample_size+1))

# Set random seed to ensure reproducibility
random.seed(42)

for col in measures:

    # Identify appropriate dictionary of keys and values
    if col in labels:
        label_dict = labels[col]
    # Exceptions for time on social media columns
    elif col == 'media_interact':
        data[col] = random.choices(np.arange(0, 101), k=sample_size)
    elif col == 'media_browse':
        data[col] = 100 - data['media_interact']
    elif col == 'media total':
        data[col] = data['media_interact'] + data['media_browse']

    # Randomly sample with replacement from the possible values for gender
    data[col] = random.choices(list(label_dict.keys()), k=sample_size)

In [9]:
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen,peer_talk_helpful,peer_talk_if,accept_peer,year_group,fsm,sen,ethnicity,english_additional,school
1,4,2,6,3,2,1,2,1,2,5,...,4,2,1,1,10,0,0,1,0,5
2,1,2,1,3,3,2,3,8,4,2,...,3,3,1,2,10,0,0,1,0,4
3,2,3,4,1,1,1,1,1,5,4,...,3,3,4,1,10,0,0,2,0,5
4,2,5,5,2,2,2,1,3,1,2,...,4,2,2,3,10,0,0,2,0,7
5,5,3,4,1,1,3,3,2,5,2,...,2,2,2,1,8,0,0,2,1,2


### Add a random MSOA for each pupil

Based on all the MSOA in Northern Devon (but in reality anticipate some regions less likely, depending on the schools participating v.s. not)

In [10]:
# Takes 28s to run - instead just import ND shapefile previously created
'''
# Import MSOA shapefile
shp_full = gpd.read_file(os.path.join(paths.data, paths.msoa_shp))

# Filter to MSOA in Northern Devon
shp_nd = shp_full[shp_full['MSOA21NM'].str.contains('North Devon|Torridge')]
shp_nd['MSOA21NM']

# Save to shapefile
shp_nd.to_file(f'{os.getcwd()}/data/msoa_nd_2021/msoa_nd_2021.shp', driver='ESRI Shapefile')
'''

# Import the Northern Devon shapefile
shp_nd = gpd.read_file(os.path.join(paths.data, paths.nd_msoa_shp))

# Show the MSOA present in this file
display(shp_nd['MSOA21NM'])

# Randomly choose an MSOA for each pupil
data['msoa'] = random.choices(list(shp_nd['MSOA21NM']), k=sample_size)

# Preview the dataframe
data.head()

0     North Devon 001
1     North Devon 002
2     North Devon 003
3     North Devon 004
4     North Devon 005
5     North Devon 006
6     North Devon 007
7     North Devon 008
8     North Devon 009
9     North Devon 010
10    North Devon 011
11    North Devon 012
12    North Devon 013
13    North Devon 014
14       Torridge 001
15       Torridge 002
16       Torridge 003
17       Torridge 004
18       Torridge 005
19       Torridge 006
20       Torridge 007
21       Torridge 008
22       Torridge 009
Name: MSOA21NM, dtype: object

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_helpful,peer_talk_if,accept_peer,year_group,fsm,sen,ethnicity,english_additional,school,msoa
1,4,2,6,3,2,1,2,1,2,5,...,2,1,1,10,0,0,1,0,5,North Devon 006
2,1,2,1,3,3,2,3,8,4,2,...,3,1,2,10,0,0,1,0,4,North Devon 006
3,2,3,4,1,1,1,1,1,5,4,...,3,4,1,10,0,0,2,0,5,North Devon 014
4,2,5,5,2,2,2,1,3,1,2,...,2,2,3,10,0,0,2,0,7,North Devon 009
5,5,3,4,1,1,3,3,2,5,2,...,2,2,1,8,0,0,2,1,2,Torridge 003


### Input some random missing data

Have chosen to exclude school from this - there should not be any pupils with no school (else there is a problem with data processing at some point, but shouldn't be possible).

In [11]:
# Set random seed to ensure reproducibility
random.seed(42)

# For each of the columns except school...
for col in data.drop('school', axis=1).columns:

    # Randomly choose how many missing data
    n_missing = random.randint(0, 100)

    # Drop that number of values, setting as NaN
    data[col] = data[col].sample(n=len(data) - n_missing)

### Input some non-random missing data

I want to introduce some scenarios to check they are being managed appropriately, such as a school not having any of a certain year group.

In [12]:
# Set school 6/F to only have Year 8s
data.loc[(data['school'] == 6) & (data['year_group'] == 10), 'year_group'] = 8
data.loc[data['school'] == 6, 'year_group'].value_counts(dropna=False)

year_group
8.0    102
NaN      8
Name: count, dtype: int64

In [13]:
# Set school 2/B to have no SEN
data.loc[(data['school'] == 2) & (data['sen'] == 1), 'sen'] = 0
data.loc[data['school'] == 2, 'sen'].value_counts(dropna=False)

sen
0.0    125
NaN      7
Name: count, dtype: int64

## Create score columns

In [14]:
# Use function to calculate scores for each pupil
data = calculate_scores(data)

# Preview the dataframe
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,home_relationship_score,home_happy_score,local_env_score,discrim_score,belong_local_score,wealth_score,future_score,climate_score,social_score,bully_score
1,4.0,2.0,6.0,3.0,2.0,1.0,2.0,1.0,2.0,5.0,...,9.0,3.0,19.000000,0.0,NaN,NaN,NaN,3.0,NaN,NaN
2,1.0,2.0,1.0,3.0,3.0,2.0,3.0,8.0,4.0,2.0,...,7.0,0.0,NaN,0.0,2.0,NaN,NaN,1.0,12.0,8.0
3,2.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,...,NaN,4.0,14.000000,0.0,4.0,0.0,NaN,1.0,NaN,9.0
4,2.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,14.0,9.0,NaN,0.0,3.0,NaN,9.0,3.0,12.0,7.0
5,5.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,13.0,9.0,16.333333,0.0,1.0,1.0,9.0,3.0,NaN,6.0


## Create label columns

This utilises the labels dictionary that we imported above.

In [15]:
# Define columns that you don't want to make labels for
exclude_col = np.append(
    ['media_interact', 'media_browse', 'media_total', 'msoa'],
    data.columns[data.columns.str.endswith('_score')].values)

# For each column, find the label dictionary
for column in data.columns.drop(exclude_col):
    label_dict = labels[column]
    # Replace the column values based on the labelling dictionary
    data[column + '_lab'] = data[column].map(label_dict)

# Preview the dataframe
display(data.head())

# Show the example of media and it's labels (which is one where the numeric
# version is not intuitive)
display(data[['media_hours', 'media_hours_lab']].drop_duplicates())

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
1,4.0,2.0,6.0,3.0,2.0,1.0,2.0,1.0,2.0,5.0,...,Fully,Somewhat helpful,Very uncomfortable,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School E
2,1.0,2.0,1.0,3.0,3.0,2.0,3.0,8.0,4.0,2.0,...,Mostly,Very helpful,Very uncomfortable,Slightly,Year 10,NaN,NaN,Ethnic minority,NaN,School D
3,2.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,...,Mostly,Very helpful,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E
4,2.0,5.0,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,Fully,Somewhat helpful,Uncomfortable,Mostly,Year 10,NaN,Non-SEN,White British,No,School G
5,5.0,3.0,4.0,1.0,1.0,3.0,3.0,2.0,5.0,2.0,...,Slightly,Somewhat helpful,Uncomfortable,Not at all,NaN,Non-FSM,Non-SEN,White British,Yes,School B


,media_hours,media_hours_lab
1,5.0,3 to 4 hours
2,3.0,1 to 2 hours
3,7.0,5 to 6 hours
7,2.0,Less than 1 hour
8,6.0,4 to 5 hours
11,4.0,2 to 3 hours
13,8.0,6 to 7 hours
14,1.0,None
20,9.0,7 hours or more
203,NaN,NaN


## Save data

In [16]:
data.to_csv(os.path.join(paths.data, paths.synthetic_data), index=False)